In [26]:
import pandas as pd
import numpy as np

filelist = ['birth.csv', 'breed.csv', 'report.csv', 'spec.csv','submission.csv']

# Read File

In [27]:
birth = pd.read_csv('./data/' + filelist[0])
breed = pd.read_csv('./data/' + filelist[1])
report = pd.read_csv('./data/' + filelist[2])
spec = pd.read_csv('./data/' + filelist[3])
submission = pd.read_csv('./data/' + filelist[4])
submission

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,ID,1
0,480,NaN
1,742,NaN
2,743,NaN
3,744,NaN
4,745,NaN
...,...,...
4258,37481,NaN
4259,37482,NaN
4260,37483,NaN
4261,37484,NaN


In [28]:
birth_index = ['乳牛編號', '分娩日期', '乾乳日期','犢牛編號1','犢牛編號2','母牛體重','登錄日期','計算胎次','胎次','分娩難易度','犢牛體型','犢牛性別','酪農場代號']
report_index = ['ID','資料年度','資料月份','酪農場代號','乳牛編號','父親精液編號','母親乳牛編號','出生日期','胎次','泌乳天數','乳量','最近分娩日期','採樣日期','月齡','檢測日期','最後配種日期','最後配種精液','配種次數','前次分娩日期','第一次配種日期','第一次配種精液']
breed_index = ['乳牛編號','配種日期','配種精液','登錄日期','孕檢','配種方式','精液種類','酪農場代號']
spec_index = ['乳牛編號','狀況類別','狀況代號','狀況日期','備註','登錄日期','酪農場代號']
birth.columns = birth_index
report.columns = report_index
breed.columns = breed_index
spec.columns = spec_index

In [29]:
# birth.describe(include = 'O')

In [30]:
def augFeatures(data, name):
    data[name] = pd.to_datetime(data[name])
    data[name+"年"] = data[name].dt.year
    data[name+'月'] = data[name].dt.month
    data[name+'日'] = data[name].dt.day
#     train[name+'星期'] = train[name].dt.dayofweek
    return data

In [31]:
# report = augFeatures(report, '出生日期')

In [32]:
# report.head()

In [33]:
# report[['泌乳天數', '乳量']].groupby(['泌乳天數'], as_index=False).mean().sort_values(by='乳量', ascending=False)

In [34]:
Train_ID = [rep_val for rep_val in report['ID'].values if rep_val not in submission['ID'].values]
Train_report = report.loc[report['ID'].isin(Train_ID)]
Train_Number = Train_report['乳牛編號'].values

In [35]:
Test_ID = [rep_val for rep_val in report['ID'].values if rep_val in submission['ID'].values]
Test_report = report.loc[report['ID'].isin(Test_ID)]
Test_Number = Test_report['乳牛編號'].values

In [36]:
report_drop_columns = ['資料年度', '資料月份', '父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Test_report_drop_columns = ['資料年度','資料月份','父親精液編號','母親乳牛編號','出生日期','最近分娩日期','採樣日期','檢測日期'
               ,'最後配種日期','最後配種精液','前次分娩日期','第一次配種日期','第一次配種精液', '酪農場代號']
Train_report_x = Train_report.drop(columns=report_drop_columns, axis = 1)

Test_report_x = Test_report.drop(columns=Test_report_drop_columns, axis = 1)
Train_report_x

,ID,乳牛編號,胎次,泌乳天數,乳量,月齡,配種次數
0,1,52612,1,78.0,29.4,26,1
1,2,52612,1,107.0,27.8,27,1
2,3,52612,1,140.0,9.0,28,2
3,4,52612,1,175.0,13.0,29,2
4,5,52612,1,210.0,13.2,30,2
...,...,...,...,...,...,...,...
37512,37513,93184912,8,12.0,20.0,130,0
37513,37514,93184912,8,137.0,21.0,134,0
37514,37515,93184912,8,168.0,19.0,135,0
37515,37516,93184912,8,197.0,12.0,136,0


In [37]:
def buildtable(data):
    t1 = data
    t2 = data
    t3 = data
    # Join
    t4 = pd.merge(t1, t2, on = ['乳牛編號', '胎次'], suffixes = ['_L', '_R'])
    # Filter
    t5 = t4[(t4.月齡_L >= t4.月齡_R)]
    # Group by
    t6 = t5.groupby(['ID_L'], as_index = False)
    # Aggregate
    t7 = t6.agg({'ID_R': 'count'})
    
    t8 = t7.rename(columns = {'ID_L': 'ID', 'ID_R': '序列號'})
    t9 = pd.merge(t8, t3, on = ['ID'])
    
    t10 = t9[['ID','乳牛編號', '胎次','乳量','序列號']]
    t10.sort_values(by = ['乳牛編號','胎次','序列號'], inplace = True)
    return t10

In [38]:
train_ID = [val[0] for val in Train_report[['ID','資料年度']].values if val[1] != 2018] 
valid_ID = [val[0] for val in Train_report[['ID','資料年度']].values if val[1] == 2018] 

In [39]:
train_r = Train_report.loc[Train_report['ID'].isin(train_ID)]
valid_r = Train_report.loc[Train_report['ID'].isin(valid_ID)]

In [40]:
print(len(train_r), len(valid_r))

26095 7159


In [41]:
train_x = buildtable(train_r)
valid_x = buildtable(valid_r)

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [42]:
# pd.set_option('display.max_columns', 37517, 'display.max_rows', 37517)

In [43]:
print(train_x.shape)
# print(train_x.head(30))

(26095, 5)


In [51]:
def build_data(train, pastMonth=5, futureMonth=1):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureMonth-pastMonth):
        past, future = train.iloc[i:i+pastMonth], train.iloc[i+pastMonth:i+pastMonth+futureMonth]["乳量"]
        for k in range(pastMonth):
            if past.iloc[k]['序列號']+(pastMonth-k-1) != past.iloc[-1]['序列號']:
                for d in past.columns: 
                    if d != '序列號': 
                        past.iloc[k, past.columns.get_loc(d)] = -1
        X_train.append(np.array(past))
        
        for k in range(futureMonth):
            if train.iloc[i+pastMonth+k]['序列號'] != past.iloc[-1]['序列號']+k+1: 
                future[k] = -1
        Y_train.append(np.array(future.values))
    return np.array(X_train), np.array(Y_train)

In [52]:
lstm_x, lstm_y = build_data(train_x.iloc[:20]) 

D:\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


In [53]:
print(lstm_x.shape, lstm_y.shape)

(14, 5, 5) (14,)


In [54]:
print(lstm_y[0][0])

15.7


# Model

In [55]:
# # Import the Keras libraries and packages
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
# from keras.layers import Dropout

# class LSTM_Model():
#     def __init__(self, time_stamp = 5):
#         model = Sequential()
#         # Adding the first LSTM layer and some Dropout regularisation
#         model.add(LSTM(units = 50, return_sequences = True, input_shape = (time_stamp, 1)))
#         model.add(Dropout(0.2))
        
#         # Adding a second LSTM layer and some Dropout regularisation
#         model.add(LSTM(units = 50, return_sequences = True))
#         model.add(Dropout(0.2))

#         # Adding a third LSTM layer and some Dropout regularisation
#         model.add(LSTM(units = 50, return_sequences = True))
#         model.add(Dropout(0.2))

#         # Adding a fourth LSTM layer and some Dropout regularisation
#         model.add(LSTM(units = 50))
#         model.add(Dropout(0.2))
        
#         # Adding the output layer
#         model.add(Dense(units = 1))
#         self.model = model
        
#     def get_model(self):
#         return self.model

In [56]:
# lstmm = LSTM_Model(lstm_x.shape[1])
# model = lstmm.get_model()

# # Compiling
# model.compile(optimizer = 'adam', loss = 'mean_squared_error')
# model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 5, 50)             10400     
_________________________________________________________________
dropout_8 (Dropout)          (None, 5, 50)             0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 5, 50)             20200     
_________________________________________________________________
dropout_9 (Dropout)          (None, 5, 50)             0         
_________________________________________________________________
lstm_10 (LSTM)               (None, 5, 50)             20200     
_________________________________________________________________
dropout_10 (Dropout)         (None, 5, 50)             0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 50)               

In [57]:
# # 進行訓練
# model.fit(lstm_x, lstm_y, epochs = 100, batch_size = 32)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).